<a href="https://colab.research.google.com/github/edsonlcandido/gemini-cartola-rodada6/blob/main/rodada6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>